In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, brier_score_loss, mean_squared_error
from sklearn.calibration import CalibrationDisplay
import numpy as np
import matplotlib.pyplot as plt


In [2]:

# Assuming `model` is your trained model


# %%
YEARS = [2018, 2019, 2020, 2021, 2022, 2023,2024]

# %%
data_all = pd.DataFrame()

def calculate_seconds(row):
    if row['qtr'] != 5:
        return 3600 - row['game_seconds_remaining']
    else:
        return 600 - row['game_seconds_remaining'] + 3600


def get_quarter_value(dataf):
    if 'END QUARTER' in dataf['desc']:
        return dataf['level_0']
    else:
        return None

for i in YEARS:  
    i_data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(i) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data_all = pd.concat([data_all,i_data])

ppr = 1

data = data_all.loc[data_all.season_type=='REG']
#data = data_all.loc[(data_all.play_type.isin(['no_play','pass','run'])) & (data_all.epa.isna()==False)]
#data.loc[data['pass']==1, 'play_type'] = 'pass'
#data.loc[data.rush==1, 'play_type'] = 'run'
data.reset_index(drop=True, inplace=True)
data['turnover'] = data['interception'] + data['fumble_lost']
data = data.dropna(subset=['posteam'])
data['inside_10'] = (data['yardline_100'] < 10).astype(int)
data['20+_play'] = (data['yards_gained'] > 19).astype(int)
#data['short_pass'] = (data['air_yards'] < 10).astype(int)
#data['medium_pass'] = ((data['air_yards'] > 9)&(data['air_yards']<20)).astype(int)
#data['deep_pass'] = (data['air_yards'] > 19).astype(int)
#data['end_zone_t'] = (data['yardline_100'] - data['air_yards']) <= 0
data['fantasy_points'] = (
    data['touchdown'] * 6 +           # 6 points per touchdown
    data['yards_gained'] * 0.1        # 0.1 points per yard gained
)
#data['distance_to_EZ_after_target'] = data['yardline_100'] - data['air_yards']


C:\Users\src15\AppData\Local\Temp\ipykernel_1320\3147829628.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_1320\3147829628.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_1320\3147829628.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

In [3]:
def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [4]:
    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]


    # derive implied team total from betting market data
    data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
    data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)

    # Use list comprehension with zip for more efficient row-wise operations
    data['implied_posteam_total'] = [
    total_finder(has_ball, home_number, away_number)
        for has_ball, home_number, away_number in zip(data['posteam_type'], data['home_implied_total'], data['away_implied_total'])
    ]

    

C:\Users\src15\AppData\Local\Temp\ipykernel_1320\3013416712.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_1320\3013416712.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_1320\3013416712.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

In [ ]:
    runs = data[data['rush']==1]
    
    df = runs[['rusher_player_name','rusher_player_id','posteam','rush','run_location','game_id','inside_10','yardline_100','ydstogo','yards_gained','fantasy_points','rush_touchdown','down','week','season','home_implied_total','away_implied_total','posteam_type']]


In [6]:


# Prepare the features (X) and target (y)
predictors = [
    'yardline_100', 'ydstogo',
    'down','run_location']

X = df[predictors]
y = df['rush_touchdown']

# Convert categorical variables (if needed)
X = pd.get_dummies(X, columns=['run_location'], drop_first=True)

# Handle missing values if any
X.fillna(X.mean(), inplace=True)  # Example for filling with mean


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create XGBoost regression model with probability-focused objective
model = xgb.XGBClassifier(
    objective='binary:logistic',  # Binary classification
    eval_metric=['logloss', 'auc'],
    use_label_encoder=False
)


# Hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 3]
}

# Grid search with both MSE and ROC-AUC scoring
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=['neg_mean_squared_error', 'roc_auc'],
    refit='neg_mean_squared_error',  # Choose MSE as primary metric
    cv=5
)
grid_search.fit(X_train, y_train)

# Best model
touchdown_model = grid_search.best_estimator_

# Make predictions
y_pred = touchdown_model.predict(X_test)

# Evaluate both regression and classification metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
roc_auc = roc_auc_score(y_test, y_pred)

print("Model Evaluation:")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"RMSE: {rmse:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

# Check calibration across probability ranges
prob_ranges = np.linspace(0, 1, 11)
print("\nProbability Calibration Check:")
for i in range(len(prob_ranges)-1):
    mask = (y_pred >= prob_ranges[i]) & (y_pred < prob_ranges[i+1])
    if mask.any():
        avg_pred_prob = y_pred[mask].mean()
        actual_prob = y_test[mask].mean()
        n_samples = mask.sum()
        print(f"\nProbability range {prob_ranges[i]:.1f}-{prob_ranges[i+1]:.1f} (n={n_samples}):")
        print(f"Average predicted: {avg_pred_prob:.3f}")
        print(f"Actual observed: {actual_prob:.3f}")

# Feature importance
importance = touchdown_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance_df)

# Example prediction function
def predict_touchdown_probability(model, play_data):
    """
    Make touchdown probability predictions for new plays.
    """
    # Ensure play_data has same preprocessing as training data
    play_data = pd.get_dummies(play_data, columns=['pass_location'], drop_first=True)
    
    # Add any missing columns from training data
    for col in model.feature_names_in_:
        if col not in play_data.columns:
            play_data[col] = 0
            
    # Reorder columns to match training data
    play_data = play_data[model.feature_names_in_]
    
    # Get predictions and ensure they're in [0,1]
    probabilities = model.predict(play_data)
    probabilities = np.clip(probabilities, 0, 1)
    
    return probabilities


# Print top factors influencing this prediction
def explain_prediction(model, play_data, feature_importance_df):
    """Explain what factors most influenced a specific prediction"""
    # Get the top 5 most important features
    top_features = feature_importance_df.head()
    
    print("\nTop factors influencing this prediction:")
    for _, row in top_features.iterrows():
        feature = row['Feature']
        if feature in play_data.columns:
            value = play_data[feature].iloc[0]
            print(f"{feature}: {value} (importance: {row['Importance']:.3f})")



c:\Users\src15\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:10:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\src15\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:10:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\src15\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:10:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\src15\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:10:11] WARNING: C:\actio

Model Evaluation:
Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.8}
RMSE: 0.1810
ROC-AUC Score: 0.6657

Probability Calibration Check:

Probability range 0.0-0.1 (n=18021):
Average predicted: 0.000
Actual observed: 0.024

Feature Importance:
               Feature  Importance
0         yardline_100    0.877591
1              ydstogo    0.064229
2                 down    0.029745
3  run_location_middle    0.017473
4   run_location_right    0.010961


c:\Users\src15\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:12:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [7]:
import pickle


with open('rushing_touchdown_model.pkl', 'wb') as file:
    pickle.dump(touchdown_model, file)